# Get video from RTV SLO MMC

I will try to create appliclation that downloads the video if I pass in the video code

First I will get the response, afterwards I will create string of the URL with video or avdio(mp3) file.

I will download the file and save it to my documents.

I will also create UI using pyQt5. :)

The `JSON` library has two main methods:

- `dumps` -- Takes in a Python object, and converts it to a string
- `loads` -- Takes a JSON string, and converts it to a Python object

In [1]:
import requests
import json
import os

# Testing

In [2]:
video_ID_number = 174511173

In [3]:
get_info_api = "http://api.rtvslo.si/ava/getRecording/{0}?client_id=82013fb3a531d5414f478747c1aca622".format(video_ID_number)

### Some of possible responses

The request we just made returned a status code of 200. Web servers return status codes every time they receive an API request. A status code provides information about what happened with a request. Here are some codes that are relevant to GET requests:

- ``200`` - Everything went okay, and the server returned a result (if any).
- `301` - The server is redirecting you to a different endpoint. This can happen when a company switches domain names, or an endpoint's name has changed.
- `401` - The server thinks you're not authenticated. This happens when you don't send the right credentials to access an API (we'll talk about this in a later mission).
- `400` - The server thinks you made a bad request. This can happen when you don't send the information the API requires to process your request, among other things.
- `403` - The resource you're trying to access is forbidden; you don't have the right permissions to see it.
- `404` - The server didn't find the resource you tried to access.

In [10]:
#get response from the web site (information)
response = requests.get(get_info_api)
print(response.status_code)
print(type(response))

200
<class 'requests.models.Response'>


In [63]:
#transform the response to the dict using json library
response_dict = response.json()

#info about the file name
file_name = response_dict['response']['mediaFiles'][0]['filename']
print(file_name)

#where the file is saved.. have to know in order to download the file
http_streamer = response_dict['response']['mediaFiles'][0]['streamers']['http']
print(http_streamer)

#check the file size, if it is too big download it in chunks and ask user if it agreese
file_size = response_dict['response']['mediaFiles'][0]['filesize']
print('File size is: {0:.0f} Bytes'.format(int(file_size)/8.))
print('File size is: {0:.2f} MB'.format(float(file_size)/2**20))

#create informative file_name and appropriate file structure
broadcast_date = response_dict['response']['broadcastDate']
print(broadcast_date.split(' ')[0])#.replace('-' ,''))

stub = response_dict['response']['stub']
video_title = response_dict['response']['title']
ending = file_name.split('.')[-1]

my_filename = broadcast_date.split(' ')[0] +' ' + stub + ' ' + video_title + '.' + ending
my_filename

2017/12/29/RadioGA-GAMMC.PR1.20171229.5.1010.mp3
http://videoweb2.rtvslo.si/ava_archive04
File size is: 6214080 Bytes
File size is: 47.41 MB
2017-12-29


'2017-12-29 radio-ga-ga Radio GA - GA.mp3'

In [5]:
file_name = '2017/12/29/RadioGA-GAMMC.PR1.20171229.5.1010.mp3'

In [6]:
http_streamer = 'http://videoweb2.rtvslo.si/ava_archive04'

In [61]:
r = requests.get(http_streamer + '/' + file_name)
print(r.status_code)

200


In [34]:
with open('./'  + my_filename, "wb") as code:
    code.write(r.content)

# Function that downloads the video

based on above testing ... and so on

In [2]:
def download_video_rtv_slo(video_ID_number, home = './', max_file_size_MB = 100):
    '''downloads video from rtv slo site
       it checks teh file size and create appropriate'''
    #this is acquired from 'https://zmaga.com/content.php?id=7526' forum
    get_info_api = "http://api.rtvslo.si/ava/getRecording/{0}?client_id=82013fb3a531d5414f478747c1aca622".format(video_ID_number)
    
    #get response from the web site (information)
    response = requests.get(get_info_api)
    if response.status_code != 200:
        return False, 'Error at acquiring the info about the video.'
    
    #transform the response to the dict using json library
    response_dict = response.json()

    #info about the file name
    file_name = response_dict['response']['mediaFiles'][0]['filename']
    print(file_name)

    #where the file is saved.. have to know in order to download the file
    http_streamer = response_dict['response']['mediaFiles'][0]['streamers']['http']
    print(http_streamer)

    #check the file size, if it is too big download it in chunks and ask user if it agreese
    file_size = response_dict['response']['mediaFiles'][0]['filesize']
    file_size_MB = float(file_size)/2**20
    print('File size is: {0:.0f} Bytes'.format(int(file_size)/8.))
    print('File size is: {0:.2f} MB'.format(file_size_MB))
    
    if file_size_MB > max_file_size_MB:
        return False, 'File size is too big, it is above {0}'.format(max_file_size_MB)

    #create informative file_name and appropriate file structure
    broadcast_date = response_dict['response']['broadcastDate']
    print(broadcast_date.split(' ')[0])#.replace('-' ,''))

    stub = response_dict['response']['stub']
    video_title = response_dict['response']['title']
    ending = file_name.split('.')[-1]

    new_filename = broadcast_date.split(' ')[0] +' ' + stub + ' ' + video_title + '.' + ending
    
    #creating the directory where to put file, home mam definiran zgoraj da lahko manjam
    save_directory = home + stub + '/'
    if not os.path.exists(save_directory):
        print('Had to create directory.')
        os.makedirs(save_directory)
    
    # get actual file
    r = requests.get(http_streamer + '/' + file_name)
    if r.status_code != 200:
        return False, "Could not load actual video file."
    try:
        with open(save_directory  + new_filename, "wb") as file:
            file.write(r.content)
    except Exception as e:
        msg = 'There was something wrong with writing to the file.'
        return e, msg
    
    msg = 'Successfully downloaded the video "{0}" which is located at "{1}"'.format(video_title, save_directory)
    return True, msg

In [76]:
download_video_rtv_slo(174496432)

2017/10/13/Politikakleeplazenja3603719.mp3
http://videoweb2.rtvslo.si/ava_archive04
File size is: 1276464 Bytes
File size is: 9.74 MB
2017-10-13
Had to create directory.


(True,
 'Successfully downloaded the video Politika klečeplazenja which is located at ./petkova-centrifuga/')

# Using above defined function

I will use above function to load desired videos, currently I am saving them in tha directory where this jupyter notebook is located.

In [85]:
# Loaded on 2018
centrifuge = [174496432,174497858,174499253,174500352,174501697,174503172,
              174504576,174505979,174510191,174507349,174508722,174511238]
radio_ga_ga = [174508592,174510079,174511173]
zapisi_iz_mocvirja = [174497146,174498583,174499845,174500969,174502444,
                      174503824,174505244,174506608,174508049,174509334,174510634]

In [80]:
for video_ID in centrifuge:
    download_video_rtv_slo(video_ID)

2017/10/13/Politikakleeplazenja3603719.mp3
http://videoweb2.rtvslo.si/ava_archive04
File size is: 1276464 Bytes
File size is: 9.74 MB
2017-10-13
2017/10/20/Paletavsebinskihnemaniev3612871.mp3
http://videoweb2.rtvslo.si/ava_archive04
File size is: 1380528 Bytes
File size is: 10.53 MB
2017-10-20
2017/10/27/KatalonciKitajciJaponciinSlovenci3621565.mp3
http://videoweb2.rtvslo.si/ava_archive04
File size is: 1242624 Bytes
File size is: 9.48 MB
2017-10-27
2017/11/03/Poltisoletnidolgsamemusebi3627089.mp3
http://videoweb2.rtvslo.si/ava_archive04
File size is: 1150224 Bytes
File size is: 8.78 MB
2017-11-03
2017/11/10/Deelaskrivalnic3634249.mp3
http://videoweb2.rtvslo.si/ava_archive04
File size is: 1235568 Bytes
File size is: 9.43 MB
2017-11-10
2017/11/17/Sprejemamosamoangele3643771.mp3
http://videoweb2.rtvslo.si/ava_archive04
File size is: 1133808 Bytes
File size is: 8.65 MB
2017-11-17
2017/11/24/Poljekrvikiganobenaobsodbanemoreoistiti3651340.mp3
http://videoweb2.rtvslo.si/ava_archive04
File siz

In [81]:
for video_ID in radio_ga_ga:
    download_video_rtv_slo(video_ID)

2017/12/15/RadioGA-GAMMC.PR1.20171215.5.1010.mp3
http://videoweb2.rtvslo.si/ava_archive04
File size is: 6088512 Bytes
File size is: 46.45 MB
2017-12-15
Had to create directory.
2017/12/22/RadioGA-GAMMC.PR1.20171222.5.1010.mp3
http://videoweb2.rtvslo.si/ava_archive04
File size is: 6386064 Bytes
File size is: 48.72 MB
2017-12-22
2017/12/29/RadioGA-GAMMC.PR1.20171229.5.1010.mp3
http://videoweb2.rtvslo.si/ava_archive04
File size is: 6214080 Bytes
File size is: 47.41 MB
2017-12-29


In [86]:
for video_ID in zapisi_iz_mocvirja:
    succ, msg = download_video_rtv_slo(video_ID)
    print(msg)

2017/10/17/20303608986.mp3
http://videoweb2.rtvslo.si/ava_archive04
File size is: 961200 Bytes
File size is: 7.33 MB
2017-10-17
Successfully downloaded the video "2030" which is located at "./zapisi-iz-mocvirja/"
2017/10/24/Prekletstvolepenedelje3617472.mp3
http://videoweb2.rtvslo.si/ava_archive04
File size is: 970752 Bytes
File size is: 7.41 MB
2017-10-24
Successfully downloaded the video "Prekletstvo lepe nedelje" which is located at "./zapisi-iz-mocvirja/"
2017/10/31/ezvodoskaekjerjemost3624583.mp3
http://videoweb2.rtvslo.si/ava_archive04
File size is: 835392 Bytes
File size is: 6.37 MB
2017-10-31
Successfully downloaded the video "Čez vodo skače, kjer je most" which is located at "./zapisi-iz-mocvirja/"
2017/11/07/Zidarskamodanapolitinihdogodkih3630096.mp3
http://videoweb2.rtvslo.si/ava_archive04
File size is: 985872 Bytes
File size is: 7.52 MB
2017-11-07
Successfully downloaded the video "Zidarska moda na političnih dogodkih" which is located at "./zapisi-iz-mocvirja/"
2017/11/14/

In [4]:
mix_14_01_2018 = [174512169,174512296,174512830,174513473,174513567]
for video_ID in mix_14_01_2018:
    download_video_rtv_slo(video_ID)

2018/01/05/RadioGA-GAMMC.PR1.20180105.5.1010.mp3
http://videoweb2.rtvslo.si/ava_archive04
File size is: 6039504 Bytes
File size is: 46.08 MB
2018-01-05
2018/01/05/Enostranskostnasvehstraneh3696088.mp3
http://videoweb2.rtvslo.si/ava_archive04
File size is: 1092144 Bytes
File size is: 8.33 MB
2018-01-05
2018/01/09/DostitekoineinvitaminaC3698991.mp3
http://videoweb2.rtvslo.si/ava_archive04
File size is: 980640 Bytes
File size is: 7.48 MB
2018-01-09
2018/01/12/RadioGA-GAMMC.PR1.20180112.5.1010.mp3
http://videoweb2.rtvslo.si/ava_archive04
File size is: 6150432 Bytes
File size is: 46.92 MB
2018-01-12
2018/01/12/Izmerljivainneizmerljivakoda3704403.mp3
http://videoweb2.rtvslo.si/ava_archive04
File size is: 1368432 Bytes
File size is: 10.44 MB
2018-01-12
